<a href="https://colab.research.google.com/github/LYZAcell/data_study/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%80%EB%84%90/%EB%8D%B0%EC%A0%80)%EC%9C%84%EA%B2%BD%EB%8F%84_%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터저널 위경도 추출
- 노션 './전처리위경도전.csv' 업로드 후 시작

## 라이브러리 설치 및 import

In [5]:
pip install requests

In [6]:
import pandas as pd
import requests

## 카카오맵 api 응답확인

In [7]:
# 카카오맵 호출되는지 확인
# api 키 권한오류 시 코랩, 로컬 등 링크 허용 필요

def test_address(address):
  # restapi키
  api_key = "ec7cbcebaedfbf302ed77da39ea7ef3b"
  headers = {"Authorization": f"KakaoAK {api_key}"}

  # API 호출 URL
  url = "https://dapi.kakao.com/v2/local/search/address.json"

  # 요청 파라미터
  params = {"query": address}

  # API 요청
  response = requests.get(url, headers=headers, params=params)

  if response.status_code == 200:
      result = response.json()
      if result['documents']:
          # 위도와 경도를 추출
          x = result['documents'][0]['x']  # 경도
          y = result['documents'][0]['y']  # 위도
          return x, y
      else:
          return None
  else:
      print(f"Error {response.status_code}: {response.text}")
      return None


In [9]:
# 사용 예제
address = "서울특별시 강남구 테헤란로 152"
coordinates = test_address(address)

if coordinates:
    print(f"주소: {address}")
    print(f"위도: {coordinates[1]}")
    print(f"경도: {coordinates[0]}")
else:
    print("좌표를 찾을 수 없습니다.")

주소: 서울특별시 강남구 테헤란로 152
위도: 37.5000242405515
경도: 127.036508620542


## df 호출

In [4]:
origin_df = pd.read_csv('./전처리위경도전.csv', encoding = 'cp949')
df = origin_df.copy()
df.head()

,시도명,시군구명,폐교명,폐교연도,학교급구분명,활용현황구분명,건물연면적,소재지도로명주소,소재지지번주소
0,강원특별자치도,강릉시,왕산초 고단분교장,2010,초등학교,대부,771.00,NaN,강원특별자치도 강릉시 왕산면 고단리 32-18
1,강원특별자치도,강릉시,옥계초 남양분교장,2011,초등학교,대부,892.00,NaN,강원특별자치도 강릉시 옥계면 남양리 528
2,강원특별자치도,양양군,양양여자중학교,2019,중학교,자체활용,10566.93,NaN,강원특별자치도 양양군 양양읍 서문리 224
3,강원특별자치도,원주시,문막초 취병분교장,2014,초등학교,대부,539.58,NaN,강원특별자치도 원주시 문막읍 취병리 358
4,부산광역시,부산진구,중앙초등학교,2008,초등학교,자체활용,5178.00,부산광역시 부산진구 전포대로 209번길 26,NaN
